In [23]:
from outils import *
from gensim.models import KeyedVectors
from sklearn.linear_model import LogisticRegression
import pydot_ng as pydot
#import matplotlib.pyplot as plt 
import gensim
import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'
import keras
from keras.layers import Dense, Dropout, Activation, Embedding, LSTM, Bidirectional, GRU
from keras.models import Sequential
from keras.metrics import binary_accuracy
from keras.callbacks import *
from keras.layers import Dense, Input, Dropout, BatchNormalization
from keras.models import Model
from keras.layers.convolutional import MaxPooling1D, Conv1D
from keras.layers import Flatten
from keras.callbacks import *
from keras.utils import plot_model  
from keras.models import load_model
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score
import pickle
import os
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import re
import numpy as np

In [39]:
x_train, y_train, x_test, y_test = load_data()

In [42]:
x_test = x_test.reshape(72000)
x_train = x_train.reshape(168000)
np.place(y_train, y_train=='male', 1)
np.place(y_train, y_train=='female', 0)
np.place(y_test, y_test=='male', 1)
np.place(y_test, y_test=='female',0 )
y_test = y_test.reshape(72000)
y_train = y_train.reshape(168000)

array(['female', 'female', 'female', ..., 'male', 'male', 'male'], dtype=object)

In [45]:
print(len(y_train))
print(len(y_test))

168000
72000


In [46]:
embedding_dim = 300
max_num_words = 100000
max_sequence_length = 30
embedding_path = './data/embeddings/wiki.ar.vec'
embedding_path = './data/embeddings/wiki_corpus_tweets_xtrain_word2vec_size300_skip_gram_wv_iter=10'
print('Tokenizing texts')
x_train, x_test, word_index = prepare_tokenized_data((x_train, x_test), max_num_words, max_sequence_length)
print('Finished tokenizing texts')
print('-' * 80)

Tokenizing texts
Loaded tokenizer.pkl
Found 144388 unique 1-gram tokens.
Min sequence length: 0
Average sequence length: 9
Max sequence length: 60
Finished tokenizing texts
--------------------------------------------------------------------------------


In [47]:
embedding_matrix = load_embedding_matrix(embedding_path, word_index, embedding_dim,gensim=gensim)

Loaded embedding matrix


In [48]:
from keras import backend as K
K.clear_session()

In [54]:
K.clear_session()
model_bi_gru = Sequential()
model_bi_gru.add(Embedding(len(word_index) + 1,
                        embedding_dim,
                        weights=[embedding_matrix],
                        input_length=max_sequence_length,
                        trainable=False))
model_bi_gru.add(Bidirectional(LSTM(150,activation='linear')))
model_bi_gru.add(Activation('relu'))
model_bi_gru.add(Dropout(0.5))
model_bi_gru.add(Dense(32,activation='relu'))
model_bi_gru.add(Dense(1, activation='sigmoid'))
# Compile model
model_bi_gru.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# checkpoint
#bi_gru_filepath="./saved_models/ASTD.Bi.GRU.model-{val_acc:.2f}.hdf5"
bi_gru_filepath="./Bi.LSTM.model-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(bi_gru_filepath, monitor='val_acc', verbose=2, save_best_only=True, mode='max')
#tbCallBack = TensorBoard(log_dir='./logs', histogram_freq=1, write_graph=True, write_images=False,embeddings_freq = 1 ,
                               # embeddings_layer_names = None ,
                               # embeddings_metadata = None)
#history2 = model.fit(x_train,y_train,batch_size=100,epochs=10,validation_data=(x_test,y_test),verbose=1,callbacks=[checkpoint])

In [55]:
history = model_bi_gru.fit(x_train,y_train,batch_size=100,epochs=5,validation_data=(x_test,y_test),verbose=1)

Train on 168000 samples, validate on 72000 samples
Epoch 1/5
168000/168000 [==============================] - 384s 2ms/step - loss: 0.6126 - acc: 0.6658 - val_loss: 0.6164 - val_acc: 0.6652
Epoch 2/5
168000/168000 [==============================] - 383s 2ms/step - loss: 0.5874 - acc: 0.6859 - val_loss: 0.6252 - val_acc: 0.6568
Epoch 3/5
168000/168000 [==============================] - 382s 2ms/step - loss: 0.5741 - acc: 0.6958 - val_loss: 0.6249 - val_acc: 0.6562: 2s - loss: 0.5741 - acc - ETA: 0s - loss: 0.5740 - acc: 0.6
Epoch 4/5
168000/168000 [==============================] - 382s 2ms/step - loss: 0.5599 - acc: 0.7058 - val_loss: 0.6253 - val_acc: 0.6540
Epoch 5/5
 28500/168000 [====>.........................] - ETA: 4:35 - loss: 0.5448 - acc: 0.7168

KeyboardInterrupt: 